In [22]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import pickle
#import tensorflow as tf

from tqdm import tqdm_notebook
from scipy.spatial.distance import hamming, cosine

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

%matplotlib inline

W0426 11:36:31.166184 139786741401408 deprecation.py:323] From /home/mark/.conda/envs/Springboard/lib/python3.6/site-packages/tensorflow_core/python/compat/v2_compat.py:65: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [7]:
def image_loader(image_path, image_size):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    image = cv2.resize(image, image_size, cv2.INTER_CUBIC)
    return image

In [8]:
def dataset_preprocessing(dataset_path, labels_file_path, image_size, image_paths_pickle):
    '''
    Loads images and labels from dataset folder.
    
    :param dataset_path: String, path to the train/test dataset folder
    :param labels_file_path: String, path to the .txt file where classes names are written
    :param image_size: tuple, single image size
    :param image_paths_pickle: String, name of a pickle file where all image paths will be saved
    '''
    
    with open(labels_file_path, 'r') as f:
        classes = f.read().split('\n')[:-1]
        
    
    images = []
    labels = []
    image_paths = []
    
    for image_name in os.listdir(dataset_path):
        try:
            image_path = os.path.join(dataset_path, image_name)
            images.append(image_loader(image_path, image_size))
            image_paths.append(image_path)
            for idx in range(len(classes)):
                if classes[idx] in image_name: #Example: 0_frog.png
                    labels.append(idx)
        except:
            pass
    
    with open(image_paths_pickle + ".pickle", 'wb') as f:
        pickle.dump(image_paths, f)
    
    assert len(images) == len(labels)
    return np.array(images), np.array(labels)

In [9]:
images, labels = dataset_preprocessing("../cifar_image_processing/cifar/train/", "../cifar_image_processing/cifar/labels.txt", (32,32), "training_images_pickle")

In [10]:
images.shape

(50000, 32, 32, 3)

In [11]:
def cosine_difference(training_set_vectors, query_vector, top_n=50):
    distances = []
    for i in range(len(training_set_vectors)):
        distances.append(cosine(training_set_vectors[i], query_vector[0]))
    
    return np.argsort(distances)[:top_n]

In [12]:
def hamming_distance(training_set_vectors, query_vector, top_n=50):
    distances = []
    for i in range(len(training_set_vectors)):
        distances.append(hamming(training_set_vectors[i], query_vector[0]))
    
    return np.argsort(distances)[:top_n]

In [13]:
def sparse_accuracy(true_labels, predicted_labels):
    
    assert len(true_labels) == len(predicted_labels)
    
    correct = 0
    for i in range(len(true_labels)):
        if np.argmax(predicted_labels[i]) == true_labels[i]:
            correct += 1
    return correct / len(true_labels)

In [14]:
def model_inputs(image_size):
    inputs = tf.placeholder(dtype=tf.float32, shape=[None, image_size[0], image_size[1],3], name='images')
    targets = tf.placeholder(dtype=tf.int32, shape=[None], name='targets')
    dropout_rate = tf.placeholder(dtype=tf.float32, name='dropout_rate')
    
    return inputs, targets, dropout_rate

In [15]:
def conv_block(inputs, 
               number_of_filters, 
               kernel_size, 
               strides=(1,1), 
               padding='SAME', 
               activation=tf.nn.relu, 
               max_pool=True, 
               batch_norm=True):
    conv_features = layer = tf.layers.conv2d(inputs=inputs,
                                            filters=number_of_filters,
                                            kernel_size=kernel_size,
                                            strides=strides,
                                            padding=padding,
                                            activation=activation)
    
    if max_pool:
        layer = tf.layers.max_pooling2d(layer,
                                       pool_size=(2,2),
                                       strides=(2,2),
                                       padding='SAME')
    if batch_norm:
        layer = tf.layers.batch_normalization(layer)
    
    return layer, conv_features

In [26]:
def dense_block(inputs,
               units,
               activation=tf.nn.relu,
               dropout_rate=None,
               batch_norm=True):
    dense_features = layer = tf.layers.dense(inputs=inputs,
                                            units=units,
                                            activation=activation)
    if dropout_rate is not None:
        layer = tf.layers.dropout(layer,
                                 rate=dropout_rate)
    if batch_norm:
        layer = tf.layers.batch_normalization(layer)
    
    return layer, dense_features

In [17]:
def opt_loss(logits,
            targets,
            learning_rate):
    
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=targets,logits=logits))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    
    return loss, optimizer

In [28]:
class ImageSearchModel(object):
    def __init__(self, 
                 learning_rate, 
                 image_size, 
                 number_of_classes=10):
        
        tf.reset_default_graph()
        
        self.inputs, self.targets, self.dropout_rate = model_inputs(image_size)
        
        normalized_images = tf.layers.batch_normalization(self.inputs)
        
        #Conv_block 1
        conv_block_1, self.conv_1_features = conv_block(inputs=normalized_images,
                                                       number_of_filters=64,
                                                       kernel_size=(3,3),
                                                       strides=(1,1),
                                                       padding='SAME',
                                                       activation=tf.nn.relu,
                                                       max_pool=True,
                                                       batch_norm=True)
        
        #Conv_block 2
        conv_block_2, self.conv_2_features = conv_block(inputs=conv_block_1,
                                                       number_of_filters=128,
                                                       kernel_size=(3,3),
                                                       strides=(1,1),
                                                       padding='SAME',
                                                       activation=tf.nn.relu,
                                                       max_pool=True,
                                                       batch_norm=True)
        
        #Conv_block 3
        conv_block_3, self.conv_3_features = conv_block(inputs=conv_block_2,
                                                       number_of_filters=256,
                                                       kernel_size=(5,5),
                                                       strides=(1,1),
                                                       padding='SAME',
                                                       activation=tf.nn.relu,
                                                       max_pool=True,
                                                       batch_norm=True)
        
        #Conv_block 4
        conv_block_4, self.conv_4_features = conv_block(inputs=conv_block_3,
                                                       number_of_filters=512,
                                                       kernel_size=(5,5),
                                                       strides=(1,1),
                                                       padding='SAME',
                                                       activation=tf.nn.relu,
                                                       max_pool=True,
                                                       batch_norm=True)
        
        #Flatten
        flat_layer = tf.layers.flatten(conv_block_4)
        
        #Dense_block_1
        dense_block_1, self.dense_1_features = dense_block(flat_layer,
                                                               units=128,
                                                               activation=tf.nn.relu,
                                                               dropout_rate=self.dropout_rate,
                                                               batch_norm=True)
        
        #Dense_block_2
        dense_block_2, self.dense_2_features = dense_block(dense_block_1,
                                                               units=256,
                                                               activation=tf.nn.relu,
                                                               dropout_rate=self.dropout_rate,
                                                               batch_norm=True)
        
        #Dense_block_3
        dense_block_3, self.dense_3_features = dense_block(dense_block_2,
                                                               units=512,
                                                               activation=tf.nn.relu,
                                                               dropout_rate=self.dropout_rate,
                                                               batch_norm=True)

        #Dense_block_4
        dense_block_4, self.dense_4_features = dense_block(dense_block_3,
                                                               units=1024,
                                                               activation=tf.nn.relu,
                                                               dropout_rate=self.dropout_rate,
                                                               batch_norm=True)
        
        logits = tf.layers.dense(inputs=dense_block_4,
                                units=number_of_classes,
                                activation=None)
        
        self.predictions = tf.nn.softmax(logits)
        
        self.loss, self.opt = opt_loss(logits=logits,
                                      targets=self.targets,
                                      learning_rate=learning_rate)

In [29]:
model = ImageSearchModel(0.001,(32,32), 10)

In [51]:
def train(model,
         epochs,
         drop_rate,
         batch_size,
         data,
         save_dir,
         save_delta=0.15):
    X_train, y_train, X_test, y_test = data
    
    #Session
    session = tf.Session()
    session.run(tf.global_variables_initializer())
    
    saver = tf.train.Saver()
    
    best_test_accuracy = 0
    
    #Training loop
    for epoch in range(epochs):
        
        train_accuracy = []
        train_loss = []
        
        for ii in tqdm_notebook(range(len(X_train) // batch_size)):
            start_id = ii * batch_size
            end_id = start_id + batch_size
            
            X_batch = X_train[start_id:end_id]
            y_batch = y_train[start_id:end_id]
            
            feed_dict = { model.inputs:X_batch,
                          model.targets:y_batch,
                          model.dropout_rate:drop_rate}
            
            #Optimize
            _, t_loss, preds_t = session.run([model.opt, model.loss, model.predictions], feed_dict=feed_dict)
            
            train_accuracy.append(sparse_accuracy(y_batch, preds_t))
            train_loss.append(t_loss)
            
        print("Epoch: {}/{}".format(epoch, epochs),  
              " | Training accuracy: {}".format(np.mean(train_accuracy)), 
              " | Training loss: {}".format(np.mean(train_loss)) )
        
        test_accuracy =[]
        
        for ii in tqdm_notebook(range(len(X_test) // batch_size)):
            start_id = ii * batch_size
            end_id = start_id + batch_size
            
            X_batch = X_test[start_id:end_id]
            y_batch = y_test[start_id:end_id]
            
            feed_dict = { model.inputs:X_batch,
                          model.dropout_rate:0.0}
            
            #Optimize
            preds_test = session.run(model.predictions, feed_dict=feed_dict)
            
            test_accuracy.append(sparse_accuracy(y_batch, preds_test))
            
        print("Test accuracy: {}".format(np.mean(test_accuracy)))
        #Saving model
        if np.mean(train_accuracy) > np.mean(test_accuracy):
            if np.abs(np.mean(train_accuracy) - np.mean(test_accuracy)) <= save_delta:
                best_test_accuracy = np.mean(test_accuracy)
                saver.save(session, "{}/model_epoch_{}.ckpt".format(save_dir, epoch))
        
    session.close()

In [45]:
epochs = 20
batch_size = 128
learning_rate = 0.001
dropout_probs = 0.6
image_size = (32,32)

In [37]:
X_train, y_train = dataset_preprocessing("../cifar_image_processing/cifar/train/", "../cifar_image_processing/cifar/labels.txt", image_size=image_size, image_paths_pickle="train_images_pickle")

In [39]:
X_train.shape

(50000, 32, 32, 3)

In [40]:
X_test, y_test = dataset_preprocessing("../cifar_image_processing/cifar/test/", "../cifar_image_processing/cifar/labels.txt", image_size=image_size, image_paths_pickle="test_images_pickle")

In [41]:
X_test.shape

(10000, 32, 32, 3)

In [42]:
model = ImageSearchModel(learning_rate, image_size)

In [43]:
data = (X_train, y_train, X_test, y_test)

In [52]:
train(model, epochs, dropout_probs, batch_size, data, 'saver')

/home/mark/.conda/envs/Springboard/lib/python3.6/site-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Epoch: 0/20  | Training accuracy: 0.3918669871794872  | Training loss: 1.7114248275756836


/home/mark/.conda/envs/Springboard/lib/python3.6/site-packages/ipykernel_launcher.py:47: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Test accuracy: 0.5404647435897436



Epoch: 1/20  | Training accuracy: 0.5748998397435897  | Training loss: 1.1782501935958862


...


Test accuracy: 0.6467347756410257



Epoch: 18/20  | Training accuracy: 0.963661858974359  | Training loss: 0.11217833310365677



Test accuracy: 0.6503405448717948



Epoch: 19/20  | Training accuracy: 0.9673677884615385  | Training loss: 0.1036379486322403



Test accuracy: 0.6617588141025641


In [53]:
def create_training_set_vectors(model, X_train, y_train, batch_size, checkpoint_path, image_size, distance='hamming'):
    session = tf.Session()
    session.run(tf.global_variables_initializer())
    
    saver = tf.train.Saver()
    saver.restore(session, checkpoint_path)
    
    dense_2_features = []
    dense_4_features = []
    
    for ii in tqdm_notebook(range(len(X_train) // batch_size)):
        start_id = ii * batch_size
        end_id = start_id + batch_size
        feed_dict = { model.inputs:X_batch, model.dropout_rate:0.0}
        
        dense_2, dense_4 = session.run([model.dense_2_features, model.dense_4_features], feed_dict=feed_dict)
        
        dense_2_features.append(dense_2)
        dense_4_features.append(dense_4)
    
    dense_2_features = np.vstack(dense_2_features)
    dense_4_features = np.vstack(dense_4_features)
    
    if distance == 'hamming':
        dense_2_features = np.where(dense_2_features < 0.5, 0, 1)
        dense_4_features = np.where(dense_4_features < 0.5, 0, 1)
        
        training_features = np.hstack((dense_2_features, dense_4_features))
        
        with open("hamming_train_vectors.pickle", "wb") as f:
            pickle.dump(training_vectors, f)
            
    elif distance == 'cosine':
        training_features = np.hstack((dense_2_features, dense_4_features))
        
        with open("cosine_train_vectors.pickle", "wb") as f:
            pickle.dump(training_vectors, f)
    

In [54]:
model = ImageSearchModel(learning_rate, image_size)

In [58]:
create_training_set_vectors(model, X_train, y_train, batch_size, "saver/model_epoch_5.ckpt", image_size)

/home/mark/.conda/envs/Springboard/lib/python3.6/site-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


NameError: name 'X_batch' is not defined

In [59]:
def simple_inference(model, session, train_set_vectors, uploaded_image_path, image_size, distance='hamming'):
    image = image_loader(uploaded_image_path, image_size)
    
    feed_dict = {model.inputs:[image], model.dropout_rate:0.0}
    
    dense_2_features, dense_4_features = session.run([model.dense_2_features, model.dense_4_features], feed_dict=feed_dict)
    
    closest_ids = None
    if distance == 'hamming':
        dense_2_features = np.where(dense_2_features < 0.5, 0, 1)
        dense_4_features = np.where(dense_4_features < 0.5, 0, 1)
        
        uploaded_image_vector = np.hstack((dense_2_features, dense_4_features))
        
        closest_ids = hamming_distance(train_set_vectors, uploaded_image_vector)
    
    elif distance == 'cosine':

        uploaded_image_vector = np.hstack((dense_2_features, dense_4_features))
        
        closest_ids = cosine_distance(train_set_vectors, uploaded_image_vector)
    
    return closest_ids